In [1]:
import pandas as pd
import numpy as np
import random
import baselines as b
import re
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, average_precision_score, balanced_accuracy_score

In [2]:
#path='./data/SAT_Predictions1/'
# path='./data/MRA_Files/'
path='./data/GPT2_Predictions/'
#path='./data/SAT_findings/'


def reports_cleaning(reports):
    
    new_reports=[]
    
    for r in list(reports):
        #print(r)
        new_reports.append(re.sub("\n|\r", " ", r))
    
    
    return new_reports
        


In [3]:
sample=[1,2,3,4,5]

bleu1,bleu2,bleu3,bleu4,M, R, C=[],[],[],[],[],[],[]

accuracy,precision,recall_mac,recall_mic=[],[],[],[]

classes=['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices']

test_orig=pd.read_csv(path+'gpt2_pred_labeled_'+str(1)+'.csv')
gt_orig=pd.read_csv(path+'gpt2_ref_labeled_'+str(1)+'.csv')

test_orig['Reports']=reports_cleaning(test_orig['Reports'])
gt_orig['Reports']=reports_cleaning(gt_orig['Reports'])


for i in tqdm(range(0,1000)):
    
    
    print('sample:', i)
    test=test_orig.copy()
    gt=gt_orig.copy()
    #gt=pd.read_csv(path+'gts_labelled'+str(1)+'.csv')
    #test=pd.read_csv(path+'res_labelled'+str(1)+'.csv')

    #gt=pd.read_csv(path+'ref_labelled'+str(i)+'.csv')
    
    
    
    lst=range(480)


    indx=random.choices(lst,k=480)
    
#     test=pd.read_csv(path+'score_labelled'+str(1)+'.csv')
#     gt=pd.read_csv(path+'score_ref_labelled'+str(1)+'.csv')

#     test=pd.read_csv(path+'gpt2_pred_labeled_'+str(1)+'.csv')
#     gt=pd.read_csv(path+'gpt2_ref_labeled_'+str(1)+'.csv')
    
#     #print(gt.shape)
    
#     test=pd.read_csv(path+'score_labelled'+str(i)+'.csv')
    
#     print(test.shape)
    
    baselines=pd.read_csv('./data/baselines_latest.csv')
    
    #print(baselines.iloc[:,0])
    test=baselines.iloc[2,:]

    test=pd.DataFrame(test).T

    test=pd.concat([test]*gt.shape[0], ignore_index=True)

    gt=gt.iloc[indx,:].reset_index(drop=True)

    test=test.iloc[indx,:].reset_index(drop=True)
    
    #test=test.sample(frac=1).reset_index(drop=True)
    
    
    gt=gt.fillna(0.0)


    gt=gt.replace(-1,0)

    test=test.fillna(0.0)


    test=test.replace(-1,0)
    
    
    data=gt.iloc[:,1:]
    
    test_data=test.iloc[:,1:]

    #test_data=test_data.sample(frac=1)
#     n=test_data.shape[0]
#     r=pd.DataFrame(data.iloc[report_id,:]).T

    pred = test_data#r * 1.0
    
    
    #### accuracy=======================================

    accuracy.append(accuracy_score(data.to_numpy().flatten(),test_data.to_numpy().flatten()))

    #print(accuracy)
    
  
   
    
#     print(is_normal)
    #test_shuffles=test.sample(frac=1)
    
    print(gt.shape,test.shape)
#     print((test.iloc[0,0]))
#     print(reports_cleaning(gt.iloc[0,0]))
#     if isinstance(gt.iloc[0,0], str):
#         print ("ordinary string")
#     elif isinstance(gt.iloc[0,0], unicode):
#         print ("unicode string")
#     else:
#         print ("not a string")
    bleus,meteor, rouge, ciders=b.all_scores((gt.iloc[:,0]),(test.iloc[:,0]))
    
#     #bleus,meteor, rouge, ciders=b.all_scores(np.array(gt.iloc[:,0])[is_normal], test.iloc[2,0])
    
    bleu1.append(bleus[0])
    bleu2.append(bleus[1])
    bleu3.append(bleus[2])
    bleu4.append(bleus[3])
    M.append(meteor)
    R.append(rouge)
    C.append(ciders)
    
data=pd.DataFrame([])

data['b1']=bleu1
data['b2']=bleu2
data['b3']=bleu3
data['b4']=bleu4
data['M']=M
data['R']=R
data['C']=C
data['accuracy']=accuracy

#print(np.mean(accuracy))
data.to_csv('samples_gpt2_base3.csv')

  0%|          | 0/1000 [00:00<?, ?it/s]

sample: 0
(480, 15) (480, 15)
{'testlen': 21600, 'reflen': 18272, 'guess': [21600, 21120, 20640, 20160], 'correct': [7066, 2188, 696, 275]}
ratio: 1.182136602451774


  0%|          | 1/1000 [00:10<2:47:28, 10.06s/it]

sample: 1
(480, 15) (480, 15)
{'testlen': 21600, 'reflen': 18610, 'guess': [21600, 21120, 20640, 20160], 'correct': [7344, 2408, 920, 479]}
ratio: 1.160666308436262


  0%|          | 2/1000 [00:20<2:51:13, 10.29s/it]

...

100%|█████████▉| 998/1000 [3:47:53<00:32, 16.28s/it]

sample: 998
(480, 15) (480, 15)
{'testlen': 21600, 'reflen': 18879, 'guess': [21600, 21120, 20640, 20160], 'correct': [7212, 2184, 690, 275]}
ratio: 1.1441283966311169


100%|█████████▉| 999/1000 [3:48:09<00:16, 16.08s/it]

sample: 999
(480, 15) (480, 15)
{'testlen': 21600, 'reflen': 18617, 'guess': [21600, 21120, 20640, 20160], 'correct': [7342, 2416, 887, 448]}
ratio: 1.1602298974055347


100%|██████████| 1000/1000 [3:48:27<00:00, 13.71s/it]
